This notebook analyzes data from https://github.com/HuthLab/deep-fMRI-dataset. To set up, see instructions in the `deep-fMRI-dataset` folder.

In [1]:
%load_ext autoreload
%autoreload 2
import datasets
import numpy as np
from os.path import join
from encoding.ridge_utils.SemanticModel import SemanticModel
from matplotlib import pyplot as plt
from typing import List
from sklearn.linear_model import RidgeCV, LogisticRegressionCV
from sklearn.feature_extraction.text import CountVectorizer
from encoding.feature_spaces import em_data_dir, data_dir, results_dir
from collections import defaultdict
import pandas as pd
import os
from fit_linear_models import get_dsets

# Load results

In [4]:
df.reset_index()

,feats,acc,feats_dim
0,eng1000vecs,0.648,985
1,eng1000fmri,0.653,1912
2,bow,0.734,16920


In [29]:
save_dir = '/home/chansingh/mntv1/deep-fMRI/results/linear_models'
dfs = []
fnames = os.listdir(save_dir)[::-1]
for fname in fnames:
    df = pd.read_pickle(join(save_dir, fname))
    print(fname)
    display(df)
    dfs.append(df.reset_index())

tweet_eval.pkl


,acc,feats_dim
feats,,
eng1000vecs,0.648,985
eng1000fmri,0.653,1912
bow,0.734,16920


trec.pkl


,acc,feats_dim
feats,,
eng1000vecs,0.734,985
eng1000fmri,0.710,1912
bow,0.858,8410


sst2.pkl


,acc,feats_dim
feats,,
eng1000vecs,0.762615,985
eng1000fmri,0.752294,1912
bow,0.803899,13774


rotten_tomatoes.pkl


,acc,feats_dim
feats,,
eng1000vecs,0.733583,985
eng1000fmri,0.740150,1912
bow,0.755159,16474


emotion.pkl


,acc,feats_dim
feats,,
eng1000vecs,0.6795,985
eng1000fmri,0.6245,1912
bow,0.8910,15186


In [30]:
pd.concat(dfs, keys=list(zip(fnames, df.index)), axis=0).round(2)

feats   acc  feats_dim
tweet_eval.pkl eng1000vecs 0  eng1000vecs  0.65        985
                           1  eng1000fmri  0.65       1912
                           2          bow  0.73      16920
trec.pkl       eng1000fmri 0  eng1000vecs  0.73        985
                           1  eng1000fmri  0.71       1912
                           2          bow  0.86       8410
sst2.pkl       bow         0  eng1000vecs  0.76        985
                           1  eng1000fmri  0.75       1912
                           2          bow  0.80      13774

In [31]:
list(zip(fnames, [df.index.tolist()] * len(fnames)))

[('tweet_eval.pkl', ['eng1000vecs', 'eng1000fmri', 'bow']),
 ('trec.pkl', ['eng1000vecs', 'eng1000fmri', 'bow']),
 ('sst2.pkl', ['eng1000vecs', 'eng1000fmri', 'bow']),
 ('rotten_tomatoes.pkl', ['eng1000vecs', 'eng1000fmri', 'bow']),
 ('emotion.pkl', ['eng1000vecs', 'eng1000fmri', 'bow'])]

### Load some data

In [13]:
dataset = 'trec'
X, y, X_test, y_test = get_dsets(dataset)
print('shapes', len(X), len(X_test))
print('labels', np.unique(y), np.unique(y_test))

Using custom data configuration default
Reusing dataset trec (/home/chansingh/.cache/huggingface/datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)


  0%|          | 0/2 [00:00<?, ?it/s]

Using custom data configuration default
Reusing dataset trec (/home/chansingh/.cache/huggingface/datasets/trec/default/1.1.0/751da1ab101b8d297a3d6e9c79ee9b0173ff94c4497b75677b59b61d5467a9b9)


  0%|          | 0/2 [00:00<?, ?it/s]

shapes 5452 500
labels [0 1 2 3 4 5] [0 1 2 3 4 5]


In [ ]:
pd.DataFrame.from_dict(r).set_index('feats')

,acc,feats_dim
feats,,
eng1000fmri,0.733583,4778
